# MOVIE STUDIO ANALYSIS

## Business Understanding

##### Our company has decided to create a new movie studio, but they don’t know anything about creating movies. We have been charged with exploring what types of films are currently doing the best at the box office. We must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.Our focus will be to make suggestions for the type of movies that will be the most profitable for our new movie division.

## Data Understanding


In [14]:
# We will be using the following datasets;
1.`rt.reviews.tsv`from [rotten tomatoes](https://www.rottentomatoes.com/)
2.`im.db` from IMDB
3.`bom.movie_gross.csv`from [BOXOFFICE MOJO] (https://www.boxofficemojo.com/)

SyntaxError: invalid syntax (2426433615.py, line 2)

##### Importing libraries

In [1]:

import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats

##### Loading Datasets

In [2]:

file_path = 'data/rt.reviews.tsv'

try:
  reviews_df = pd.read_csv(file_path, sep='\t',on_bad_lines='skip',encoding='ISO-8859-1')
except pd.errors.ParserError as e:
    print(f'Error parsing file:{e}')


In [3]:
conn = sqlite3.connect('data/im.db')
q = """SELECT *
FROM movie_basics
JOIN movie_ratings
USING(movie_id)"""
imdb_df = pd.read_sql(q,conn)



In [9]:
path = 'data/bom.movie_gross.csv'
bom_df = pd.read_csv(path)
